In [1]:
from data_utils import load_data, preprocess_data_faiss, preprocess_data_faiss_test

In [2]:
data = load_data("data")

In [3]:
embeddings = preprocess_data_faiss(data)
embeddings.shape

/mnt/c/Users/Raman/PycharmProjects/stackexchange-recsys/data_utils.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts["body"] = posts["body"].map(clean_html)


(11812, 2048)

In [4]:
posts, test_data = preprocess_data_faiss_test(data)
test_data

/mnt/c/Users/Raman/PycharmProjects/stackexchange-recsys/data_utils.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts["id"] = posts["id"].astype("int32")


,postid,relatedpostid,linktypeid
0,5620,3624,1
1,5129,4967,1
2,5800,174,1
3,3418,121,1
4,3317,1815,1
...,...,...,...
4267,174,970,1
4268,10224,10194,1
4269,5464,2711,1
4270,3706,3184,1


In [5]:
import faiss
import numpy as np

In [6]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [7]:
test_embeddings = embeddings[test_data["postid"]]

In [8]:
for k in range(6, 22, 5):
    D, I = index.search(test_embeddings, k=k)

    correct = 0

    for related_post_id, recommendations in zip(test_data["relatedpostid"], I):
        if related_post_id in I:
            correct += 1

    print(f"Recall@k (k = {k - 1}): {correct / test_data.shape[0]}")

Recall@k (k = 5): 0.8625235404896422
Recall@k (k = 10): 0.9383911756793113
Recall@k (k = 15): 0.9607210115684692
Recall@k (k = 20): 0.9709443099273608


In [23]:
from sklearn.decomposition import PCA

In [30]:
pca = PCA(n_components=128)
reduced_embeddings = pca.fit_transform(embeddings)

In [31]:
index = faiss.IndexFlatL2(reduced_embeddings.shape[1])
index.add(reduced_embeddings)
test_embeddings = reduced_embeddings[test_data["postid"]]

for k in range(6, 22, 5):
    D, I = index.search(test_embeddings, k=k)

    correct = 0

    for related_post_id, recommendations in zip(test_data["relatedpostid"], I):
        if related_post_id in I:
            correct += 1

    print(f"Recall@k (k = {k - 1}): {correct / test_data.shape[0]}")

Recall@k (k = 5): 0.8810868980360506
Recall@k (k = 10): 0.9383911756793113
Recall@k (k = 15): 0.9704062415926823
Recall@k (k = 20): 0.9808985741189131


In [9]:
random_embedding = embeddings[
    np.random.choice(embeddings.shape[0])
].reshape(1, -1)

In [10]:
D, I = index.search(random_embedding, k=6)

In [11]:
print(f"Distances: {D}")
print(f"Indices: {I}")

Distances: [[0.        1.176375  1.2567656 1.3929937 1.4644415 1.4777496]]
Indices: [[ 4044  2014  3880 11395  3324   693]]


In [12]:
print(
    f"===ORIGINAL POST===\n"
    f"TITLE: {posts['title'].iloc[I[0][0]]}\n"
    f"CONTENT: {posts['body'].iloc[I[0][0]]}"
)

for i in range(1, 6):
    print(
        f"===RECOMMENDATION {i} (distance = {D[0][i]})===\n"
        f"TITLE: {posts['title'].iloc[I[0][i]]}\n"
        f"CONTENT: {posts['body'].iloc[I[0][i]]}"
    )

===ORIGINAL POST===
TITLE: How long does grease take to go bad/rancid when sitting out?
CONTENT: If I make a skillet full of bacon, how long can I leave the grease sitting out before it is unsafe/rancid?  Is the answer different for ground beef grease?

===RECOMMENDATION 1 (distance = 1.1763750314712524)===
TITLE: How long will grease (from bacon or roast) keep in the fridge?
CONTENT: I kept all of the bacon grease from making bacon this weekend, and now I have a jar of solidified grease in the fridge. How long can I use this to impart bacony goodness to my fried vegetables before it goes bad?
What is the shelf life of the fat drippings from bacon, roast and other meats?

===RECOMMENDATION 2 (distance = 1.256765604019165)===
TITLE: How long can I reuse this delicious bacon grease for cooking?
CONTENT: I cooked some bacon about a week ago and left the grease in the pan.  I've used it the last two nights with great success for cooking other meals.
How long is it safe to reuse?  I figure 